In [41]:
import joblib
import numpy as np
import pandas as pd

In [42]:
def create_features_from_timestamp(ts, hist_df, zone):
    # Lag features
    lag_1 = hist_df.at[ts - pd.Timedelta(minutes=5), zone]
    lag_24 = hist_df.at[ts - pd.Timedelta(hours=2), zone]
    lag_168 = hist_df.at[ts - pd.Timedelta(days=7), zone]

    # Time-based features (matching training logic)
    hour = ts.hour
    minute_bin = ts.minute // 5
    weekday = ts.weekday()
    month = ts.month

    features = {
        "is_weekday": int(weekday < 5),
        "hour_sin": np.sin(2 * np.pi * hour / 24),
        "hour_cos": np.cos(2 * np.pi * hour / 24),
        "bin5_sin": np.sin(2 * np.pi * minute_bin / 12),
        "bin5_cos": np.cos(2 * np.pi * minute_bin / 12),
        "dow_sin": np.sin(2 * np.pi * weekday / 7),
        "dow_cos": np.cos(2 * np.pi * weekday / 7),
        "month_sin": np.sin(2 * np.pi * month / 12),
        "month_cos": np.cos(2 * np.pi * month / 12),
        "lag_1": lag_1,
        "lag_24": lag_24,
        "lag_168": lag_168
    }

    return pd.DataFrame([features])

def make_forecast(time_step,zone_ids,historic_df,models):
    
    prediction = {}

    for zone in zone_ids:
        X = create_features_from_timestamp(time_step, historic_df, zone)
        if X is not None:
            model = models[zone]
            prediction[zone] = model.predict(X)[0]
        else:
            prediction[zone] = np.nan

    return prediction


In [43]:
# Load all processed data from Jan to Dec
processed_folder = "data/processed"
file_names = [f"processed_{i:02d}.csv" for i in range(1, 8)]

historic_df = pd.concat([
    pd.read_csv(f"{processed_folder}/{fname}", index_col=0, parse_dates=True)
    for fname in file_names
])
historic_df.index = pd.to_datetime(historic_df.index)

# Load trained models
model_folder = "data/models/xgb"
zone_ids = historic_df.columns
models = {zone: joblib.load(f"{model_folder}/xgb_zone_{zone}.joblib") for zone in zone_ids}

# Load model error distribution
error_distribution = pd.read_csv("data/models/xgb/val_error_distributions.csv",index_col=0)

In [44]:
# Run prediction loop for last week of July 2010
start_ts = pd.Timestamp("2010-07-25 00:00:00")
end_ts = pd.Timestamp("2010-07-31 23:55:00")

timestamps = pd.date_range(start=start_ts, end=end_ts, freq="5min")

In [45]:
# --- Inputs ---
time_step = timestamps[0]  # specify the timestamp
prediction = make_forecast(time_step, zone_ids, historic_df, models)  # shape: (n_zones,)

# --- Load validation error distribution ---
val_err_df = pd.read_csv("data/models/xgb/val_error_distributions.csv", index_col=0)
val_err_df.index = val_err_df.index.astype(int)  # ensure zone IDs are int

# --- Generate samples ---
n_samples = 100
samples = {}

for zone in zone_ids:
    mean_forecast = prediction[zone]
    err_mean = val_err_df.loc[int(zone), "mean"]
    err_std = val_err_df.loc[int(zone), "std"]
    
    # Sample from normal distribution centered at forecast + residual bias
    zone_samples = np.random.normal(loc=mean_forecast + err_mean, scale=err_std, size=n_samples)
    samples[zone] = zone_samples

# --- Convert to DataFrame ---
samples_df = pd.DataFrame(samples)
samples_df.index.name = "sample_id"


In [46]:
prediction

{'4': 19.603739,
 '12': 0.66072917,
 '13': 14.678369,
 '24': 8.51144,
 '41': 13.644029,
 '42': 10.202907,
 '43': 11.408259,
 '45': 4.019348,
 '48': 70.93027,
 '50': 41.64568,
 '68': 80.68585,
 '74': 13.635518,
 '75': 17.813421,
 '79': 148.42027,
 '87': 26.93339,
 '88': 6.4559917,
 '90': 39.264954,
 '100': 17.806334,
 '107': 67.07496,
 '113': 40.446453,
 '114': 50.26215,
 '116': 10.237096,
 '120': 0.1622789,
 '125': 18.916723,
 '127': 3.3276725,
 '128': 0.1517544,
 '137': 39.64742,
 '140': 26.163395,
 '141': 48.291862,
 '142': 48.35974,
 '143': 23.2319,
 '144': 36.64362,
 '148': 81.34176,
 '151': 16.240034,
 '152': 4.796307,
 '153': 0.1860685,
 '158': 57.00488,
 '161': 34.860092,
 '162': 52.357647,
 '163': 29.926594,
 '164': 45.68096,
 '166': 11.510589,
 '170': 70.41677,
 '186': 46.395634,
 '194': 0.18467309,
 '202': 2.4496255,
 '209': 4.4667797,
 '211': 22.459225,
 '224': 13.755243,
 '229': 32.47827,
 '230': 53.51555,
 '231': 38.238987,
 '232': 15.5765505,
 '233': 27.303534,
 '234': 64

In [47]:
samples_df

,4,12,13,24,41,42,43,45,48,50,...,237,238,239,243,244,246,249,261,262,263
sample_id,,,,,,,,,,,,,,,,,,,,,
0,24.850142,2.860915,8.522007,8.502523,14.594280,11.959050,14.988807,4.264792,65.909038,33.653534,...,13.617039,26.579192,39.846453,8.632734,6.765291,46.641790,53.205581,5.088280,22.893130,48.578210
1,17.565951,1.336475,14.167541,11.380345,12.736972,11.828483,11.863531,3.331291,82.787682,40.143886,...,23.078823,39.995726,53.744379,6.748363,8.913238,58.213894,70.831745,15.041559,21.488822,44.499655
2,27.988781,-0.249117,17.576883,9.691304,9.666119,11.208813,2.739418,3.462775,84.952857,45.098559,...,13.145220,14.871130,55.193644,5.261670,14.048458,51.621538,62.186198,10.133729,30.847597,53.662713
3,21.680435,-0.550048,19.421263,9.806482,14.460676,9.814398,5.735185,6.343801,76.230382,55.667971,...,15.573478,29.061816,58.706290,8.194749,10.926102,46.412023,53.832105,17.419741,23.614469,45.827631
4,19.657454,0.784818,1.947091,10.535028,16.274085,12.008391,10.868445,6.720891,63.564643,39.040741,...,15.555955,25.156589,35.903913,6.655743,10.370213,61.020307,68.969551,12.170605,29.765041,39.433507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,16.663185,1.222713,15.214530,4.326436,11.155304,10.808104,17.311418,1.735228,73.951324,34.683880,...,19.854548,34.780602,40.354799,7.882810,9.739116,51.138616,76.088804,12.727380,23.469274,43.153527
96,18.072103,1.683846,23.357945,8.746121,12.889331,6.380750,13.172825,5.354650,66.402291,45.438731,...,12.267967,44.173351,50.085995,8.940362,12.436992,45.320933,63.429441,11.017436,26.399154,55.890117
97,21.823503,0.681835,17.176269,10.868784,14.622689,7.101106,10.183774,4.590306,76.812001,45.357256,...,23.318491,34.029438,40.785376,8.205723,9.037121,38.301430,60.129590,9.737943,29.307458,58.162040


In [48]:
# --- Make point forecasts for all timestamps ---
all_predictions = {}

for time_step in timestamps:
    print(time_step)
    prediction = make_forecast(time_step, zone_ids, historic_df, models)  # returns dict: {zone_id: value}
    all_predictions[time_step] = prediction

# --- Convert to DataFrame ---
forecast_df = pd.DataFrame.from_dict(all_predictions, orient='index')  # timestamps as index, zones as columns
forecast_df.index.name = "timestamp"

forecast_df


2010-07-25 00:00:00
2010-07-25 00:05:00
2010-07-25 00:10:00
2010-07-25 00:15:00
2010-07-25 00:20:00
2010-07-25 00:25:00
2010-07-25 00:30:00
2010-07-25 00:35:00
2010-07-25 00:40:00
2010-07-25 00:45:00
2010-07-25 00:50:00
2010-07-25 00:55:00
2010-07-25 01:00:00
2010-07-25 01:05:00
2010-07-25 01:10:00
2010-07-25 01:15:00
2010-07-25 01:20:00
2010-07-25 01:25:00
2010-07-25 01:30:00
2010-07-25 01:35:00
2010-07-25 01:40:00
2010-07-25 01:45:00
2010-07-25 01:50:00
2010-07-25 01:55:00
2010-07-25 02:00:00
2010-07-25 02:05:00
2010-07-25 02:10:00
2010-07-25 02:15:00
2010-07-25 02:20:00
2010-07-25 02:25:00
2010-07-25 02:30:00
2010-07-25 02:35:00
2010-07-25 02:40:00
2010-07-25 02:45:00
2010-07-25 02:50:00
2010-07-25 02:55:00
2010-07-25 03:00:00
2010-07-25 03:05:00
2010-07-25 03:10:00
2010-07-25 03:15:00
2010-07-25 03:20:00
2010-07-25 03:25:00
2010-07-25 03:30:00
2010-07-25 03:35:00
2010-07-25 03:40:00
2010-07-25 03:45:00
2010-07-25 03:50:00
2010-07-25 03:55:00
2010-07-25 04:00:00
2010-07-25 04:05:00


,4,12,13,24,41,42,43,45,48,50,...,237,238,239,243,244,246,249,261,262,263
timestamp,,,,,,,,,,,,,,,,,,,,,
2010-07-25 00:00:00,19.603739,0.660729,14.678369,8.511440,13.644029,10.202907,11.408259,4.019348,70.930267,41.645679,...,15.188669,27.507744,40.505493,7.581620,10.486414,50.178192,66.116653,11.092414,26.419048,49.244984
2010-07-25 00:05:00,22.861313,0.754746,13.625380,9.728794,14.263759,7.111531,7.604748,4.106046,72.836334,42.649204,...,22.158304,33.820774,31.959396,7.770105,12.416259,44.391159,63.012615,10.717807,21.215248,36.059082
2010-07-25 00:10:00,22.974180,0.372286,13.658953,9.897530,13.766119,9.976248,15.211652,7.213233,74.303940,36.894394,...,19.844589,35.354000,45.285767,7.573794,11.787738,40.475510,62.597347,10.831779,27.363407,48.592724
2010-07-25 00:15:00,20.805616,0.447503,16.472610,9.973710,14.167008,8.905149,12.690852,5.093904,72.279190,31.542522,...,17.329805,29.864494,39.304413,8.618484,12.504239,43.609985,73.495758,11.646742,19.468760,42.197186
2010-07-25 00:20:00,20.850388,0.402365,15.867502,10.022715,9.799263,9.882619,10.855941,4.785038,69.253960,34.429779,...,18.773603,28.212309,36.282928,6.709179,9.134907,47.043938,60.376770,11.780223,14.669167,52.555489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-07-31 23:35:00,16.358280,0.538264,18.040157,9.326409,12.336114,9.732189,12.789776,8.013709,71.851662,40.567120,...,28.003056,31.690910,35.135010,6.837941,9.482888,45.797337,67.891579,12.437858,20.396362,45.542667
2010-07-31 23:40:00,24.477909,0.426364,19.182825,9.037587,12.744191,10.422785,12.404389,6.411638,74.312668,34.811203,...,32.268768,34.517445,36.567795,5.621479,10.181121,46.692471,61.927608,9.801574,21.826759,47.078487
2010-07-31 23:45:00,25.635847,0.422720,14.939797,8.912440,13.283604,9.715136,17.229622,6.645422,69.278267,35.552189,...,32.757748,27.468878,34.210953,5.951690,10.155823,42.398415,69.458046,9.856190,24.260050,33.430794


In [49]:
forecast_df.to_csv('data/models/xgb/forecast_df.csv')

In [50]:
actual = pd.read_csv('data/processed/processed_07.csv',index_col=0)

In [62]:
actual.to_csv('data/models/xgb/actual.csv')

In [51]:
# Ensure the index is a datetime index
actual.index = pd.to_datetime(actual.index)

# Filter rows from 25-07-2010 00:00:00 onwards
actual_df = actual[actual.index >= '2010-07-25 00:00:00']

# Display the result
actual_df


,4,12,13,24,41,42,43,45,48,50,...,237,238,239,243,244,246,249,261,262,263
2010-07-25 00:00:00,25,2,14,10,19,5,8,4,79,50,...,27,37,34,7,13,50,67,9,21,35
2010-07-25 00:05:00,30,0,14,11,13,12,19,12,86,44,...,22,38,52,6,11,45,67,11,27,48
2010-07-25 00:10:00,18,1,21,13,18,7,16,5,82,38,...,20,29,42,12,11,49,78,12,19,41
2010-07-25 00:15:00,20,0,18,11,6,12,12,5,76,43,...,21,27,39,5,9,53,67,12,13,52
2010-07-25 00:20:00,20,0,14,13,16,5,13,3,81,41,...,18,38,32,9,12,43,68,15,22,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-07-31 23:35:00,22,0,25,7,12,15,12,6,80,39,...,36,32,38,3,9,50,63,5,17,44
2010-07-31 23:40:00,31,0,9,6,14,13,21,4,69,42,...,37,25,37,4,10,43,71,5,25,30
2010-07-31 23:45:00,12,1,19,5,19,15,7,7,66,43,...,28,42,38,5,9,47,81,18,23,43
2010-07-31 23:50:00,24,1,17,4,15,19,14,3,75,50,...,39,36,51,10,10,55,59,15,17,48


In [59]:
actual

,4,12,13,24,41,42,43,45,48,50,...,237,238,239,243,244,246,249,261,262,263
2010-07-01 00:00:00,11,0,14,9,10,13,9,3,40,17,...,28,30,38,8,8,20,57,9,25,38
2010-07-01 00:05:00,15,2,15,9,14,10,9,4,45,13,...,27,36,35,9,14,22,36,15,18,28
2010-07-01 00:10:00,14,1,9,7,19,12,10,6,48,12,...,24,27,33,7,8,23,36,12,15,49
2010-07-01 00:15:00,13,1,9,6,19,12,13,4,69,10,...,23,29,20,7,5,24,40,11,22,40
2010-07-01 00:20:00,14,0,14,11,11,13,10,6,43,24,...,25,22,29,7,14,11,32,7,19,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-07-31 23:35:00,22,0,25,7,12,15,12,6,80,39,...,36,32,38,3,9,50,63,5,17,44
2010-07-31 23:40:00,31,0,9,6,14,13,21,4,69,42,...,37,25,37,4,10,43,71,5,25,30
2010-07-31 23:45:00,12,1,19,5,19,15,7,7,66,43,...,28,42,38,5,9,47,81,18,23,43
2010-07-31 23:50:00,24,1,17,4,15,19,14,3,75,50,...,39,36,51,10,10,55,59,15,17,48


In [52]:
actual_df.to_csv('data/models/xgb/actual_df.csv')

In [53]:
res = pd.read_csv('data/models/xgb/xgboost_forecast_results.csv', index_col=0)

In [54]:
res

,MAE,MSE
4,2.519789,11.334456
12,0.976487,2.210212
13,3.618990,25.573922
24,1.884120,5.905241
41,2.235987,8.271463
...,...,...
246,5.219062,50.817495
249,5.255131,50.527155
261,2.513730,10.749856
262,4.108026,29.472684


In [55]:
res.mean()

MAE     4.008041
MSE    37.686469
dtype: float64

In [57]:
err = pd.read_csv('data/models/xgb/val_error_distributions.csv', index_col=0)

In [61]:
err.to_csv('data/models/xgb/val_error_distributions.csv')